In [66]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.action_chains import ActionChains
import csv
import scrapy
import time
import re
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
from selenium.webdriver.firefox.options import Options
import multiprocessing as mp
import requests as req
import bs4
import numpy as np

In [46]:
#you will need to put your own path to the firefox binary here
#binary = FirefoxBinary()
legistar = 'https://milwaukee.legistar.com/Legislation.aspx'

def visit_site(legistar, headless=False, phantom=False):
    if phantom == False:
        if headless:
            options = Options()
            options.headless = True
            mkeDriver = webdriver.Firefox(options = options, executable_path='/Users/ayaspan/Documents/Personal/voting_records_mke/geckodriver')
        else:
            mkeDriver = webdriver.Firefox(executable_path='/Users/ayaspan/Documents/Personal/voting_records_mke/geckodriver')
        try: 
            mkeDriver.get(legistar)
        except:
            return None
        return mkeDriver
    else:
        mkeDriver = webdriver.PhantomJS()
        mkeDriver.get(legistar)
        return mkeDriver


In [375]:
mkeDriver2 = visit_site(legistar)

In [385]:
test = get_next_page(mkeDriver2)

[]


In [3]:
def select_all_years(mkeDriver):
    years = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstYears_Input")
    years.send_keys('All Years')
    years.submit()
    #return years

In [4]:
#makes it so that you have max results

def get_max_results(mkeDriver):
    step1 = mkeDriver.find_elements_by_id("ctl00_ContentPlaceHolder1_menuMain")[0]
    action = ActionChains(mkeDriver)
    action.click(step1.find_elements_by_class_name("rmItem ")[1])
    #action.click(step1.find_elements_by_class_name("rmItem ")[1].find_element_by_css_selector("a"))
    action.perform()
    time.sleep(1)
    step2 = step1.find_elements_by_class_name("rmItem ")[1].parent
    #print(step2.text)
    action = ActionChains(mkeDriver)
    # action_test2 = ActionChains(testDriver)
    action.move_to_element(step2.find_element_by_class_name("rmSlide").find_element_by_class_name("rmLast"))
    action.click(step2.find_element_by_class_name("rmSlide").find_element_by_class_name("rmLast"))
    action.perform()
    #return mkeDriver

In [5]:
# re.search(r'(?<=of\s)[0-9]+',mkeDriver.find_elements_by_class_name("rgPagerCell")[0]\
#             .find_element_by_class_name("rgInfoPart").text)[0]

In [6]:
#need to consider the recursion involved in looping through pages OR use above

In [7]:
#locats page link 
def get_next_page(mkeDriver):
    pgs_drop = mkeDriver.find_elements_by_class_name("rgPagerCell")
    #print(pgs_drop)
    try:
        return pgs_drop[0].find_element_by_xpath("//a[@class='rgCurrentPage']/following-sibling::a")
    except:
        return None

def go_to_next_page(next_page):
    next_page.click()

In [19]:
def select_ordinance(mkeDriver):
    type_ordinance = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_Arrow").click()
    legislation_type = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_DropDown")\
                .find_elements_by_class_name("rcbItem")
    print(legislation_type[15].text)
    actions = ActionChains(mkeDriver)
    actions.move_to_element(legislation_type[15])
    actions.click(legislation_type[15])
    actions.perform()
    #choose_ordinance.perform()#.double_click()
#type_legislation.s
#type_legislation.submit()

In [9]:
# legislation_type = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_DropDown")\
#                 .find_elements_by_class_name("rcbItem")
# hover_ord = ActionChains(mkeDriver).move_to_element(legislation_type[14])
# hover_ord.perform().send_keys("Ordinance").double_click()

In [10]:
def start_search(mkeDriver):
    s_button = mkeDriver.find_element_by_id("visibleSearchButton")
    s_button.click()
    #return mkeDriver

In [64]:
def get_records_table(mkeDriver):
    records_table = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_gridMain")
    r_table_body = records_table.find_elements_by_class_name("rgRow")
    r_table_body2 = records_table.find_elements_by_class_name("rgAltRow")
    return r_table_body, r_table_body2

In [12]:
# for i in records_table.find_elements_by_css_selector('a'):
#     if 'https' in i.get_attribute('href'):
#         print(i.get_attribute('href'))

In [13]:
#r_table_body = records_table.find_element_by_css_selector("tbody")

In [14]:
#can change # of records per page :)

In [63]:
#if status passed
import time

def get_meta_data(legistar):
    count=0
    leg_dict = {}
    col_names = [0,'type','status','created_date','final_action','title']
    
    #CALL OTHER REQUISITE FUNCTIONS FIRST
    mkeDriver = visit_site(legistar)
    #time.sleep(5)
    select_all_years(mkeDriver)
    time.sleep(2)
    select_ordinance(mkeDriver)
    start_search(mkeDriver)
    time.sleep(2)
    get_max_results(mkeDriver)
    
    while True:
        time.sleep(4)
        r_table_body, r_table_body2 = get_records_table(mkeDriver)
        if count in leg_dict:
            print('dup')
            continue
        else:
            leg_dict[count]= {i.find_element_by_css_selector('td').find_element_by_css_selector('a').text:{
                          'link': i.find_element_by_css_selector('a').get_attribute('href'),
                           'other_info':[j.text for j in i.find_elements_by_css_selector('td')]} for i in r_table_body}
            count+=1
            leg_dict[count] = {i.find_element_by_css_selector('td').find_element_by_css_selector('a').text:{
                          'link': i.find_element_by_css_selector('a').get_attribute('href'),
                           'other_info':[j.text for j in i.find_elements_by_css_selector('td')]} for i in r_table_body2}
        #works
        #leg_dict[count] = [i.text for i in r_table_body]
        
        #print(r_table_body.text)
        #print(r_table_body.find_elements_by_class_name('rgRow')[0].text)
#         for row in r_table_body.find_elements_by_class_name('rgRow'):
#             print(row.get_attribute('id'),'id')
#             file_num = row.find_element_by_css_selector('td').find_element_by_css_selector('a').text
#             leg_dict[file_num] = {}
#             link = row.find_element_by_id('ctl00_ContentPlaceHolder1_gridMain_ctl00_ctl04_hypFile').get_attribute('href')
#             leg_dict[file_num]['file_link'] = link
#             print(file_num,'file_num and ', link,' link')

#             for d,col in zip(row.find_elements_by_css_selector('td'),col_names):
#                 if not col:
#                     continue
#                 #if leg_dict[file_num]['status'] in ('Passed',"Mayor's Office",'Dead','Placed On File'): use this later for going through links
#                 leg_dict[file_num][col] = d.text
# #                 leg_dict[file_num]['status'] = d.text
# #                 leg_dict[file_num]['created_date'] = d.text
# #                 leg_dict[file_num]['final_action'] = d.text
#                 leg_dict[file_num]['title'] = d.text
        time.sleep(.5)
        
        
        next_page = get_next_page(mkeDriver)
        
        if not next_page:
            return leg_dict
        else:
            next_page.click()
        
        
        count+=1
    

In [60]:
#leg_dict

In [65]:
meta_legis_dict = get_meta_data(legistar)

Ordinance


In [68]:
df = pd.DataFrame()

for key,val in meta_legis_dict.items():
    df = df.append(pd.DataFrame.from_dict(val).T)


In [69]:
df.shape

(4716, 2)

In [120]:
def splitDataFrameList(df,target_column,col_list):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    row_accumulator = []

    def splitListToRows(row):
        #new_row = {}
        split_row = row[target_column]
        new_row = row.to_dict()
        for col_name, s in zip(col_list,split_row):
            new_row[col_name] = s
        row_accumulator.append(new_row)

    df.apply(splitListToRows, axis=1)
    new_df = pd.DataFrame(row_accumulator)
    return new_df

In [130]:
col_list = ['file_number','type','status','file_created','final_action','title']
new_df = splitDataFrameList(df,'other_info',col_list)

In [131]:
new_df.drop(columns='other_info',inplace=True)

In [132]:
new_df.tail()

,file_created,file_number,final_action,link,status,title,type
4711,10/30/1992,921221,5/14/2002,https://milwaukee.legistar.com/LegislationDeta...,Placed On File,An ordinance relating to use of public passeng...,Ordinance
4712,7/7/1992,920585,5/19/2000,https://milwaukee.legistar.com/LegislationDeta...,Placed On File,A substitute ordinance relating to apprentices...,Ordinance
4713,2/5/1992,911966,10/30/1998,https://milwaukee.legistar.com/LegislationDeta...,Placed On File,A substitute ordinance relating to architectur...,Ordinance
4714,11/29/1990,901402,6/25/2002,https://milwaukee.legistar.com/LegislationDeta...,Passed,A substitute ordinance relating to the prohibi...,Ordinance
4715,7/28/1987,870858,9/23/2003,https://milwaukee.legistar.com/LegislationDeta...,Placed On File,Substitute ordinance relating to the change in...,Ordinance


In [161]:
new_df.dtypes

file_created    object
file_number     object
final_action    object
link            object
status          object
title           object
type            object
dtype: object

In [133]:
new_df.to_pickle('legistar_meta_info.pkl')

In [77]:
#meta_legis_dict[2] #worked

In [153]:
# def objectIdentificationUsingWaits(self, maxTimeOut, locatorProperties, locatorType = "xpath"):
#         element = None
#         try:
#             WebDriverWait(self.driver, maxTimeOut, ignored_exceptions=[StaleElementReferenceException]).until(
#                 EC.presence_of_element_located((self.getLocatorType(locatorType), locatorProperties)))
#             element = self.driver.find_element(locatorType, locatorProperties)
#         except:
#             print("Exception occurred during object identification.")
#         return element

In [152]:
# from selenium.common.exceptions import StaleElementReferenceException

# test = visit_site(legistar)
# #time.sleep(5)
# select_all_years(test)
# time.sleep(5)
# select_ordinance(test)
# start_search(test)
# time.sleep(3)
# get_max_results(test)
# time.sleep(.5)



# col_names = [0,'type','status','created_date','final_action','title']
# leg_dict = {}
# r_table_body = get_records_table(test)
# print(r_table_body.text)
# print(r_table_body.find_elements_by_class_name('rgRow'))
# #print(r_table_body.find_elements_by_class_name('rgRow')[0].text)
# for row in r_table_body.find_elements_by_class_name('rgRow'):
#     print(row.get_attribute('id'),'id')
#     file_num = row.find_element_by_css_selector('td').find_element_by_css_selector('a').text
#     leg_dict[file_num] = {}
#     try:
#         link = row.find_element_by_id('ctl00_ContentPlaceHolder1_gridMain_ctl00_ctl04_hypFile').get_attribute('href')
#     except:
#         link='no link'
#     leg_dict[file_num]['file_link'] = link
#     print(file_num,'file_num and ', link,' link')
#     for d,col in zip(row.find_elements_by_css_selector('td'),col_names):
#                 if not col:
#                     continue
                
#                 leg_dict[file_num][col] = d.text

In [563]:
#conn = sqlite3.connect('vr_ords.db')
engine = create_engine('sqlite:///vr_ords.db', echo=False)
connection = engine.connect()
#df.to_sql('users', con=engine)

In [3]:
#new_df.to_sql('meta_data', con=engine)

In [14]:
meta_df = pd.read_sql('SELECT * FROM meta_data;', connection, index_col='index')

In [18]:
meta_df.head()

,file_created,file_number,final_action,link,status,title,type
index,,,,,,,
0,2/5/2019,181642,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to the city’s towing dro...,Ordinance
1,2/5/2019,181644,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to domestic partners of ...,Ordinance
2,2/5/2019,181619,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance to further amend the 2019 offices...,Ordinance
3,2/5/2019,181598,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to the change in zoning ...,Ordinance
4,2/5/2019,181583,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,A substitute ordinance relating to traffic con...,Ordinance


In [5]:
def check_status(rowDriver):
    status = rowDriver.find_element_by_id('ctl00_ContentPlaceHolder1_lblStatus2').text
    if len(status):
            return status
    else: return False

def check_passed(rowDriver):
    '''
    if ordinance was passed will say Passed
    '''
    if check_status(rowDriver) == 'Passed':
        return 'Passed'
    elif check_status(rowDriver) == 'Placed On File':
        return 'Placed On File - Denied'
    else:
        return False

In [4]:
def get_legis_table(mkeDriver):
    records_table = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_gridLegislation")
    r_table_body = records_table.find_elements_by_class_name("rgRow")
    r_table_body2 = records_table.find_elements_by_class_name("rgAltRow")
    return r_table_body, r_table_body2

In [103]:
from datetime import datetime

date_pattern = '%m/%d/%Y'

def get_date_time(row):
    #print(row)
    return datetime.strptime(row, '%m/%d/%Y')

def get_year(row):
    
    return row.year
#pd.datetime(meta_df[meta_df.file_created!=' '].file_created[1])

In [100]:
meta_df_post_nov2018 = meta_df[meta_df.file_created!= ' ']

In [110]:
meta_df_post_nov2018[pd.isna(meta_df.link)]

,file_created,file_number,final_action,link,status,title,type,date_time,year
index,,,,,,,,,


In [101]:
meta_df_post_nov2018['date_time'] = meta_df_post_nov2018.file_created.apply(get_date_time)
meta_df_post_nov2018['year'] = meta_df_post_nov2018.date_time.apply(get_year)

In [105]:
meta_df_post_nov2018 = meta_df_post_nov2018[meta_df_post_nov2018.date_time > get_date_time('11/08/2018')]

In [113]:
meta_df_post_nov2018.shape

(40, 9)

In [19]:
meta_df.link[0]


'https://milwaukee.legistar.com/LegislationDetail.aspx?ID=3857556&GUID=A3259554-D312-43C3-B3D9-9697F6D567B1&Options=ID|Text|&Search='

In [ ]:
#def get_passed

In [ ]:
testDrive = visit_site(meta_df.link.iloc[-2])

In [43]:
meta_df.link.iloc[-2]

'https://milwaukee.legistar.com/LegislationDetail.aspx?ID=145734&GUID=1097BAA1-1B3D-443C-9ABA-2DA545261E52&Options=ID|Text|&Search='

In [12]:
import requests as req
import bs4

records = req.get(meta_df.link.iloc[-2])

soup = bs4.BeautifulSoup(records.content,'html.parser')

In [23]:
print(meta_df.link.iloc[-2])

https://milwaukee.legistar.com/LegislationDetail.aspx?ID=145734&GUID=1097BAA1-1B3D-443C-9ABA-2DA545261E52&Options=ID|Text|&Search=


In [514]:
# def get_legis_table(mkeDriver):
#     records_table = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_gridLegislation")
#     r_table_body = records_table.find_elements_by_class_name("rgRow")
#     r_table_body2 = records_table.find_elements_by_class_name("rgAltRow")
#     return r_table_body, r_table_body2

def get_legis_table_soup(soup):
    r_table_body = soup.find(id = 'ctl00_ContentPlaceHolder1_gridLegislation').find_all('tr','rgRow')
    r_table_body2 = soup.find(id = 'ctl00_ContentPlaceHolder1_gridLegislation').find_all('tr','rgAltRow')
    return r_table_body, r_table_body2

In [531]:
def get_control_soup(soup):
    r_control = soup.find(id='ctl00_ContentPlaceHolder1_pageDetails').find(id='ctl00_ContentPlaceHolder1_hypInControlOf2')
    if r_control.has_attr('onclick'):
        link = r_control['href']
    else:
        link = None
    text = r_control.get_text()
    return text, link

In [523]:
# records = req.get(meta_df.link.iloc[-2])
# soup = bs4.BeautifulSoup(records.content,'html.parser')
# get_control_soup(soup)

In [520]:
meta_df.link.iloc[-2]

'https://milwaukee.legistar.com/LegislationDetail.aspx?ID=145734&GUID=1097BAA1-1B3D-443C-9ABA-2DA545261E52&Options=ID|Text|&Search='

In [46]:
#WORKED

table_body_test, table_body_test2 = get_legis_table_soup(meta_df.link.iloc[-2])
pre_url = 'https://milwaukee.legistar.com/'
for  ct,j in enumerate(table_body_test):
    for ct2,i in enumerate(j.find_all('td')):
    #print(i.select('a'))
        if ct2 != 6:
            print(i.get_text())
        else:
            #need ot use regex to get url extension and attach it to standard legistar pre-url
            if i.a.has_attr('onclick'):
                print(pre_url + re.search(r"\'{1}.*\',",i.a['onclick'])[0].strip('\'|,'))

7/12/2002
4

CITY CLERK

PUBLISHED
 
 
https://milwaukee.legistar.com/HistoryDetail.aspx?ID=718012&GUID=86E98FD2-FAE6-4E31-A26D-F9AC5FAE9CD5

Meeting details



Not available


6/25/2002
4

COMMON COUNCIL

PASSED
Pass
16:1
https://milwaukee.legistar.com/HistoryDetail.aspx?ID=717850&GUID=7DF3C679-287B-4523-BC2F-5067EB235FA5

Meeting details



Not available


6/13/2002
4

CITY CLERK

DRAFT SUBMITTED
 
 
https://milwaukee.legistar.com/HistoryDetail.aspx?ID=717604&GUID=1E13F7CB-3F66-4F98-8353-BE10AF9D4A29

Meeting details



Not available


6/12/2002
3

PUBLIC WORKS COMMITTEE

RECOMMENDED FOR  PASSAGE AND ASSIGNED
Pass
4:1
https://milwaukee.legistar.com/HistoryDetail.aspx?ID=717217&GUID=4C676835-BB9E-4AAD-AD9F-647C5C75B69A

Meeting details



Not available


6/6/2002
2

PUBLIC WORKS COMMITTEE

HEARING NOTICES SENT
 
 
https://milwaukee.legistar.com/HistoryDetail.aspx?ID=717361&GUID=EAB0D336-13B9-4223-B161-1C33F0580695

Meeting details



Not available


6/6/2002
2

PUBLIC WORKS COMMITTEE


In [33]:
list(table_body_test)

['\n',
 <td class="rgSorted"><font face="Tahoma" size="2">7/12/2002</font></td>,
 <td><font face="Tahoma" size="2">4</font></td>,
 <td nowrap="nowrap"><font face="Tahoma" size="2">
 <a id="ctl00_ContentPlaceHolder1_gridLegislation_ctl00_ctl04_hypBody"><font color="Black" face="Tahoma" size="2">CITY CLERK</font></a>
 </font></td>,
 <td><font face="Tahoma" size="2">PUBLISHED</font></td>,
 <td><font face="Tahoma" size="2"> </font></td>,
 <td><font face="Tahoma" size="2"> </font></td>,
 <td><font face="Tahoma" size="2">
 <a href="#" id="ctl00_ContentPlaceHolder1_gridLegislation_ctl00_ctl04_hypDetails" onclick="radopen('HistoryDetail.aspx?ID=718012&amp;GUID=86E98FD2-FAE6-4E31-A26D-F9AC5FAE9CD5', 'HistoryDetail');return false;"><font color="Blue" face="Tahoma" size="2">Action details</font></a>
 </font></td>,
 <td><font face="Tahoma" size="2">
 <a class="meeting_NoMeeting" id="ctl00_ContentPlaceHolder1_gridLegislation_ctl00_ctl04_hypMeetingDetail"><font color="Gray" face="Tahoma" size="2">Me

In [23]:
meta_df.reset_index(drop=True)

,file_created,file_number,final_action,link,status,title,type
0,2/5/2019,181642,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to the city’s towing dro...,Ordinance
1,2/5/2019,181644,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to domestic partners of ...,Ordinance
2,2/5/2019,181619,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance to further amend the 2019 offices...,Ordinance
3,2/5/2019,181598,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to the change in zoning ...,Ordinance
4,2/5/2019,181583,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,A substitute ordinance relating to traffic con...,Ordinance
5,1/14/2019,181536,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,A substitute ordinance establishing a Site Pla...,Ordinance
6,1/14/2019,181527,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,A substitute ordinance relating to a humane of...,Ordinance
7,1/14/2019,181540,2/5/2019,https://milwaukee.legistar.com/LegislationDeta...,Mayor's Office,A substitute ordinance relating to parking con...,Ordinance
8,1/14/2019,181501,2/5/2019,https://milwaukee.legistar.com/LegislationDeta...,Mayor's Office,A substitute ordinance to further amend the 20...,Ordinance
9,12/18/2018,181435,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to the issuance of parki...,Ordinance


In [502]:
meta_df.link[8]

'https://milwaukee.legistar.com/LegislationDetail.aspx?ID=3836471&GUID=CEBD6F20-27D3-4ACE-AA64-85FB988E46A6&Options=ID|Text|&Search='

In [556]:
def paralellize_get_passed(df, func, num_processes, num_splits):
    
    df_split = np.array_split(df, num_splits)
    pool = Pool(num_processes)
    
    #mess with chunksize dont really understand
    out = pool.map_async(func, df_split)
    pool.close()
    pool.join()
   
    print('test1')
    
    return out.get()
    
def mp_apply_get_dets(df):
    return df.apply(get_passed_dets_link, axis=1)
    
def get_passed_dets_link(x):
    '''
    The line item with the vote is PASSED
    '''
    #print(x)
    pre_url = 'https://milwaukee.legistar.com/'
    link = str(x['link'])
    #leg_drive = visit_site(link, headless=True,phantom=False)
    #     if not leg_drive:
    #         return None
    if not len(link):
        return None
    
    records = req.get(link)
    soup = bs4.BeautifulSoup(records.content,'html.parser')
    
    test_dict = {}
    c_text,c_link = get_control_soup(soup)
    
    r_table_body, r_table_body2 = get_legis_table_soup(soup)
    for ct, row in enumerate(r_table_body+r_table_body2):
        test_dict[ct] = {}
        row_elements = row.find_all('td')
        for k,v in zip(['Date','Version','Action_By','Action','Result','Tally','A_Details','M_Details','Video'], row_elements):
            if k == 'A_Details':
                if v.a.has_attr('onclick'):
                    test_dict[ct][k] = pre_url + re.search(r"\'{1}.*\',",v.a['onclick'])[0].strip('\'|,')    
            else:
                test_dict[ct][k] = v.get_text()
        #print(x['file_number'])
        test_dict[ct]['file_number'] = x['file_number']
    #test_dict = {k:row for k, row in test_dict.items() if row['Action'] == 'PASSED'}
    rv = False
    for k, row in test_dict.items():
        if row['Action'] == 'PASSED':
            row['In_Control'] = c_text
            if c_link:
                row['Control_Comm_Link'] = pre_url+c_link
            else:
                row['Control_Comm_Link'] = c_link
            test_dict = {k:row}
            rv = True
    #for k, row in test_dict.items():
    if rv:
        #print('test')
        return test_dict
    else:
        return None
        
        

In [ ]:
import warnings
#warnings.filterwarnings('ignore')
from multiprocessing import Pool
#links_dict = {}
#out2= []
start = time.time()
#df_split = np.array_split(meta_df, 20)
#with Pool(processes=20) as pool:
#         for i, row in enumerate(meta_df.iterrows()):
#             #print(row)
#             rowDriver = visit_site(row[1]['link'], True)
#             cv = check_passed(rowDriver)
#             if cv == 'Passed':
#                 rv = get_passed_dets_link(rowDriver)
#                 if rv:
#                     links_dict[i] = rv
out = paralellize_get_passed(meta_df, mp_apply_get_dets, 8,16)

end=time.time()

print(end-start)

In [560]:
out[0][2]

{1: {'Date': '2/26/2019',
  'Version': '1',
  'Action_By': '\nCOMMON COUNCIL\n',
  'Action': 'PASSED',
  'Result': 'Pass',
  'Tally': '14:0',
  'A_Details': 'https://milwaukee.legistar.com/HistoryDetail.aspx?ID=17286601&GUID=8D7BD1EF-3BA5-4BD1-BAEF-6837E51A92F1',
  'M_Details': '\nMeeting\xa0details\n',
  'Video': '\n\nNot\xa0available\n\n',
  'file_number': '181619',
  'In_Control': 'COMMON COUNCIL',
  'Control_Comm_Link': None}}

In [561]:
good_rows=[v  for i in out  for row in i  if row !=None for k,v in row.items()]

In [116]:
#good_rows

In [565]:
passed_df = pd.DataFrame(good_rows)

passed_df.to_sql('passed_legislation', con=engine)

In [566]:
passed_df.head()

,A_Details,Action,Action_By,Control_Comm_Link,Date,In_Control,M_Details,Result,Tally,Version,Video,file_number
0,https://milwaukee.legistar.com/HistoryDetail.a...,PASSED,\nCOMMON COUNCIL\n,None,2/26/2019,COMMON COUNCIL,\nMeeting details\n,Pass,14:0,1,\n\nNot available\n\n,181619
1,https://milwaukee.legistar.com/HistoryDetail.a...,PASSED,\nCOMMON COUNCIL\n,None,2/26/2019,COMMON COUNCIL,\nMeeting details\n,Pass,14:0,1,\n\nNot available\n\n,181583
2,https://milwaukee.legistar.com/HistoryDetail.a...,PASSED,\nCOMMON COUNCIL\n,None,2/26/2019,COMMON COUNCIL,\nMeeting details\n,Pass,14:0,1,\n\nNot available\n\n,181527
3,https://milwaukee.legistar.com/HistoryDetail.a...,PASSED,\nCOMMON COUNCIL\n,None,2/5/2019,COMMON COUNCIL,\nMeeting details\n,Pass,15:0,0,\n\nNot available\n\n,181540
4,https://milwaukee.legistar.com/HistoryDetail.a...,PASSED,\nCOMMON COUNCIL\n,None,2/5/2019,COMMON COUNCIL,\nMeeting details\n,Pass,15:0,1,\n\n\nVideo\n\n,181501


In [567]:
# from sqlalchemy import MetaData
# from sqlalchemy.ext.declarative import declarative_base

# base = declarative_base()
# metadata = MetaData(engine, reflect=True)
# table = metadata.tables.get('passed_legislation')
# base.metadata.drop_all(engine, [table], checkfirst=True)
#passed_df = pd.read_sql('DROP TABLE passed_legislation;', connection)

In [125]:
passed_df.shape

(3627, 10)

In [205]:
passed_df.head()

,A_Details,Action,Action_By,Date,M_Details,Result,Tally,Ver,Video,file_number
0,https://milwaukee.legistar.com/HistoryDetail.a...,PASSED,\nCOMMON COUNCIL\n,2/26/2019,\nMeeting details\n,Pass,14:0,1,\n\nNot available\n\n,181619
1,https://milwaukee.legistar.com/HistoryDetail.a...,PASSED,\nCOMMON COUNCIL\n,2/26/2019,\nMeeting details\n,Pass,14:0,1,\n\nNot available\n\n,181583
2,https://milwaukee.legistar.com/HistoryDetail.a...,PASSED,\nCOMMON COUNCIL\n,2/26/2019,\nMeeting details\n,Pass,14:0,1,\n\nNot available\n\n,181527
3,https://milwaukee.legistar.com/HistoryDetail.a...,PASSED,\nCOMMON COUNCIL\n,2/5/2019,\nMeeting details\n,Pass,15:0,0,\n\nNot available\n\n,181540
4,https://milwaukee.legistar.com/HistoryDetail.a...,PASSED,\nCOMMON COUNCIL\n,2/5/2019,\nMeeting details\n,Pass,15:0,1,\n\n\nVideo\n\n,181501


In [6]:
meta_df =  pd.read_sql('SELECT * FROM meta_data;', connection, index_col='index')

In [7]:
meta_df.shape

(4716, 7)

In [8]:
meta_df.head()

,file_created,file_number,final_action,link,status,title,type
index,,,,,,,
0,2/5/2019,181642,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to the city’s towing dro...,Ordinance
1,2/5/2019,181644,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to domestic partners of ...,Ordinance
2,2/5/2019,181619,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance to further amend the 2019 offices...,Ordinance
3,2/5/2019,181598,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to the change in zoning ...,Ordinance
4,2/5/2019,181583,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,A substitute ordinance relating to traffic con...,Ordinance


In [236]:
# records = req.get(passed_df['A_Details'][0])

# soup = bs4.BeautifulSoup(records.content,'html.parser')


In [480]:
def get_votes(x):
    link=x['A_Details']
    if pd.isna(link):
        return None
    records = req.get(link)
    soup = bs4.BeautifulSoup(records.content,'html.parser')
    table = soup.find(id = 'ctl00_ContentPlaceHolder1_tblVote2')
    if table == None:
        return None
    table_body1 = table.find_all('tr','rgRow') 
    table_body2 = table.find_all('tr','rgAltRow')
    vote_dict ={}
    for idx,i in enumerate(table_body1+table_body2):
    #print(i)
        vote_dict[idx]={}
        for k,v in zip(['Person','Vote'],i.find_all('td')):
            #print(k+'\n'+row.get_text())
        #         for idx, v in enumerate(row):
        #             print(j)
            text = v.get_text()
            if text == 'Aye':
                vote_dict[idx][k] = 1
            elif text in ['Nay','No']:
                vote_dict[idx][k] = -1
            elif text =='Absent':
                vote_dict[idx][k] = 0
            else:
                vote_dict[idx][k] = v.get_text().strip('\n').replace("ALD. ",'')
            vote_dict[idx]['file_number'] = x['file_number']
    return vote_dict

def mp_apply_get_votes(df):
    print('test')
    return df.apply(get_votes, axis=1)
    

In [497]:
vote_dict ={}
records = req.get(passed_df['A_Details'][3457])
soup = bs4.BeautifulSoup(records.content,'html.parser')
table_body1 = table.find_all('tr','rgRow') 
table_body2 = table.find_all('tr','rgAltRow')
table = soup.find(id = 'ctl00_ContentPlaceHolder1_tblVote2')
for idx,i in enumerate(table_body1+table_body2):
    #print(i)
    vote_dict[idx]={}
    for k,v in zip(['Person','Vote'],i.find_all('td')):
        #print(k+'\n'+row.get_text())
#         for idx, v in enumerate(row):
#             print(j)
        text = v.get_text()
        if 'ALD' not in text:
        #    vote_dict[idx][k] = 1
        #elif text in ['Nay','No']:
        #    vote_dict[idx][k] = -1
        #elif text =='Absent':
        #    vote_dict[idx][k] = 0
            vote_dict[idx][k] = text
        else:
            vote_dict[idx][k] = v.get_text().strip("\n").replace("ALD. ",'')
        vote_dict[idx]['file_number'] = idx

In [479]:
vote_dict

{0: {'Person': 'PRATT', 'file_number': 0, 'Vote': 1},
 1: {'Person': "D'AMATO", 'file_number': 1, 'Vote': 1},
 2: {'Person': 'SCHRAMM', 'file_number': 2, 'Vote': 1},
 3: {'Person': 'Fredrick Gordon', 'file_number': 3, 'Vote': 1},
 4: {'Person': 'RICHARDS', 'file_number': 4, 'Vote': 1},
 5: {'Person': 'SCHERBERT', 'file_number': 5, 'Vote': 1},
 6: {'Person': 'PAWLINSKI', 'file_number': 6, 'Vote': 1},
 7: {'Person': 'NARDELLI', 'file_number': 7, 'Vote': 1},
 8: {'Person': 'HINES JR.', 'file_number': 8, 'Vote': 1},
 9: {'Person': 'KALWITZ', 'file_number': 9, 'Vote': 1},
 10: {'Person': 'HENNINGSEN', 'file_number': 10, 'Vote': 1},
 11: {'Person': 'JOHNSON-ODOM', 'file_number': 11, 'Vote': 1},
 12: {'Person': 'FRANK', 'file_number': 12, 'Vote': 1},
 13: {'Person': 'BUTLER', 'file_number': 13, 'Vote': 1},
 14: {'Person': 'James N. Witkowiak', 'file_number': 14, 'Vote': 1},
 15: {'Person': 'BREIER', 'file_number': 15, 'Vote': 1},
 16: {'Person': 'MURPHY', 'file_number': 16, 'Vote': 1}}

In [482]:
import warnings
#warnings.filterwarnings('ignore')
from multiprocessing import Pool
#links_dict = {}
#votes = []
start = time.time()
#df_split = np.array_split(meta_df, 20)
#with Pool(processes=20) as pool:
#         for i, row in enumerate(meta_df.iterrows()):
#             #print(row)
#             rowDriver = visit_site(row[1]['link'], True)
#             cv = check_passed(rowDriver)
#             if cv == 'Passed':
#                 rv = get_passed_dets_link(rowDriver)
#                 if rv:
#                     links_dict[i] = rv
out2 = paralellize_get_passed(passed_df, mp_apply_get_votes , 7,7)
    #out2= out2.append(i)
end=time.time()

print(end-start)

test
test
test
test
test
test
test
540.9539711475372


In [483]:
pd.DataFrame(out2[0][1]).T

,Person,Vote,file_number
0,HAMILTON,1,181583
1,KOVAC,1,181583
2,DODD,1,181583
3,RAINEY,1,181583
4,LEWIS,1,181583
5,BORKOWSKI,1,181583
6,WITKOWSKI,Excused,181583
7,STAMPER,1,181583
8,JOHNSON,1,181583
9,BAUMAN,1,181583


In [495]:
# votes_df = pd.DataFrame()
# for dc in out2:
#     for row in dc:
#         if row:
#             next_row = pd.DataFrame.from_dict(row,orient='index')
#             votes_df = votes_df.append(next_row)
# for dc in out2:
#     for row in dc:
#         if row:
#             print(row.items())
votes_df = pd.DataFrame([sub for dc in out2 for row in dc if row for sub in row.values()])

In [498]:
# def votes_to_words(x):
#     if x==1:
#         return 'Aye'
#     elif x==-1:
#         return 'Nay'
#     elif x== 0:
#         return 'Absent'
#     else:
#         return x

In [499]:
#votes_df.Vote = votes_df.Vote.apply(votes_to_words)

In [500]:
votes_df.Vote.value_counts()

Aye        51971
Excused     2163
Nay          791
Abstain       28
Absent        25
Name: Vote, dtype: int64

In [379]:
test = pd.DataFrame([row for dc in out2 for row in dc if row])
test[test.file_number==181619]

,Person,Vote,file_number


In [501]:
votes_df.to_sql('votes', con=engine)

In [255]:
#passed_df.iloc[2480].A_Details

'https://milwaukee.legistar.com/HistoryDetail.aspx?ID=735895&GUID=8D40DC3B-8F24-4285-B41F-32B8D6BF50C0'